In [1]:
from pymongo import MongoClient
import json
from bson import ObjectId
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
import unicodedata
import os
import requests 

from dotenv import load_dotenv
from utils import espiadinha, get_df
load_dotenv()

True

<h3>Funções para acessar MongoDB Alexandria:  </h3>
*Utilize "help(nome da função)" para entender os atributos da função (exemplo: help(espiadinha))

In [2]:
def from_html_to_text(tag):
    soup = BeautifulSoup(tag, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()   
    text = soup.get_text()
    return text

In [3]:
#credenciais de acesso ao alexandria
server_type = os.getenv("server_type")
MONGO_ALEXANDRIA_USERNAME = os.getenv('MONGO_ALEXANDRIA_USERNAME')
MONGO_ALEXANDRIA_PASSWORD = os.getenv('MONGO_ALEXANDRIA_PASSWORD')
MONGO_ALEXANDRIA_HOST = os.getenv('MONGO_ALEXANDRIA_HOST')
str_conn = f"{server_type}://{MONGO_ALEXANDRIA_USERNAME}:{MONGO_ALEXANDRIA_PASSWORD}@{MONGO_ALEXANDRIA_HOST}" 
client = MongoClient(str_conn)
db = client['alexandria-kroton']

In [33]:
#import das bases do alexandria
cursos={
        'Gastronomia':['PLANEJAMENTO DE CARDÁPIO', 'ADMINISTRAÇÃO APLICADA À GASTRONOMIA'],
        'Engenharia Civil':['FÍSICA GERAL E EXPERIMENTAL - MECÂNICA','QUÍMICA E CIÊNCIA DOS MATERIAIS'],
        'Ciências Contábeis':['ANÁLISE DE CUSTOS', 'MÉTODOS QUANTITATIVOS'],
        'Publicidade e Propaganda':['SISTEMAS BRASILEIROS DE COMUNICAÇÃO', 'SOCIEDADE BRASILEIRA E CIDADANIA']
    }

df_disc=get_df(db,'disciplines',['title','learningUnits','krotonId'])
df_LUs=get_df(db,'learningunits',['_id','classes','krotonId'])
df_cls=get_df(db,'classes',['_id','blocks','krotonId','title'])
df_blk=get_df(db,'blocks',['_id','questions','content','title'])
df_quest=get_df(db,'questions',['_id','type','questionModel','title','description','feedback','alternatives','classId'])

Requisição de conteúdo de aula

In [46]:
sequencia=0
curso_lista=['Ciências Contábeis']
for curso in curso_lista:
    disciplina = cursos[curso][sequencia]
    disciplina = ''.join(c for c in disciplina if not unicodedata.category(c).startswith('C'))
    print(f"Requisitando curso: {curso} - Disciplina: {disciplina}")
    
    #Filtro de Unidades de ensino de uma Disciplina
    filtro_LUs = df_disc[df_disc.title.isin([disciplina])]['learningUnits'].to_list()[-1]
    
    url = f"https://sag-api-kroton.platosedu.io/api/v1/external/components/{ID_MAIA_API_ORDER}/learningUnits"
    ordem = requests.request("GET", url, data="",headers={'x_secret': os.getenv('MONGO_MAIA_PASSWORD')})
    ordem = json.loads(ordem.text)
    df_ordem = pd.DataFrame()
    for row in range(len(ordem)):
        df_ordem=pd.concat([df_ordem,pd.DataFrame([ordem[row]['id'],ordem[row]['order']]).transpose()])
    df_ordem.columns=['krotonId','ORDER']

    #Filtro e enriquecimento da base LUs com ordem de Maia
    df_LUs_filtrada = df_LUs[df_LUs._id.isin(filtro_LUs)].merge(df_ordem,how='left',on='krotonId')
    #Filtro de classes dentro de uma unidade de ensino
    filtro_LU_CLASSES_aninhado = df_LUs_filtrada.sort_values('ORDER')['classes'].to_list()
    
    #Para cada unidade de ensino vamos extrair o conteúdo de cada aula
    dict_conteudo = {}
    for LU_ORDER, filtro_CLASSES_FROM_LU in enumerate(filtro_LU_CLASSES_aninhado):
        LU_ORDER=LU_ORDER+1
        #print(f'Fazendo LU: {LU_ORDER}')
        
        #filtro assets são todos os assets de cada aula de uma unidade de ensino
        filtro_ASSETS = df_cls[df_cls._id.isin(filtro_CLASSES_FROM_LU)]['blocks'].to_list()
        dict_conteudo[f"Unidade_{LU_ORDER}"] = {}
        #
        for AULA_ORDER, ASSETS_FROM_AULA in enumerate(filtro_ASSETS):
            AULA_ORDER=AULA_ORDER+1
            #print(f'Fazendo aula: {AULA_ORDER}')
            conteudo = f"Unidade_{LU_ORDER} - AULA_{AULA_ORDER} =|> "
            for texto in df_blk[
                (df_blk._id.isin(ASSETS_FROM_AULA))
                &(df_blk.content.notna())
                &(~df_blk.title.isin(['Siga em Frente...','Referências','Saiba mais']))
                ][['title','content']].iterrows():
                conteudo=conteudo+f'{from_html_to_text(texto[1][0])}: {from_html_to_text(texto[1][1])}'
            dict_conteudo[f"Unidade_{LU_ORDER}"][f"AULA_{AULA_ORDER}"] = conteudo
    


Requisitando curso: Ciências Contábeis - Disciplina: ANÁLISE DE CUSTOS


C:\Users\renan.vital\AppData\Local\Temp\ipykernel_28648\2067502202.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conteudo=conteudo+f'{from_html_to_text(texto[1][0])}: {from_html_to_text(texto[1][1])}'
C:\Users\renan.vital\AppData\Local\Temp\ipykernel_28648\2067502202.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conteudo=conteudo+f'{from_html_to_text(texto[1][0])}: {from_html_to_text(texto[1][1])}'
C:\Users\renan.vital\AppData\Local\Temp\ipykernel_28648\2067502202.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated a

In [48]:
sequencia=0
curso_lista=['Ciências Contábeis']
for curso in curso_lista:
    disciplina = cursos[curso][sequencia]
    disciplina = ''.join(c for c in disciplina if not unicodedata.category(c).startswith('C'))
    print(f"Requisitando curso: {curso} - Disciplina: {disciplina}")
    
    #Filtro de Unidades de ensino de uma Disciplina
    filtro_LUs = df_disc[df_disc.title.isin([disciplina])]['learningUnits'].to_list()[-1]
    
    url = f"https://sag-api-kroton.platosedu.io/api/v1/external/components/{ID_MAIA_API_ORDER}/learningUnits"
    ordem = requests.request("GET", url, data="",headers={'x_secret': os.getenv('MONGO_MAIA_PASSWORD')})
    ordem = json.loads(ordem.text)
    df_ordem = pd.DataFrame()
    for row in range(len(ordem)):
        df_ordem=pd.concat([df_ordem,pd.DataFrame([ordem[row]['id'],ordem[row]['order']]).transpose()])
    df_ordem.columns=['krotonId','ORDER']

    #Filtro e enriquecimento da base LUs com ordem de Maia
    df_LUs_filtrada = df_LUs[df_LUs._id.isin(filtro_LUs)].merge(df_ordem,how='left',on='krotonId')
    #Filtro de classes dentro de uma unidade de ensino
    filtro_LU_CLASSES_aninhado = df_LUs_filtrada.sort_values('ORDER')['classes'].to_list()

    #Para cada unidade de ensino vamos extrair o conteúdo de cada aula
    dict_questoes = {}
    for LU_ORDER, filtro_CLASSES_FROM_LU in enumerate(filtro_LU_CLASSES_aninhado):
        LU_ORDER=LU_ORDER+1
        print(f'Fazendo LU: {LU_ORDER}')
        dict_questoes[f"Unidade_{LU_ORDER}"] = {}
        
        #filtro assets são todos os assets de cada aula de uma unidade de ensino
        filtro_ASSETS = df_cls[df_cls._id.isin(filtro_CLASSES_FROM_LU)]['blocks'].to_list()
        
        for AULA_ORDER, ASSETS_FROM_AULA in enumerate(filtro_ASSETS):

            for id_questions in df_blk[(df_blk._id.isin(ASSETS_FROM_AULA))&(df_blk.title.str.contains('Prova'))]['questions']:
                print(f'Fazendo aula: {AULA_ORDER+1} - {len(id_questions)} questões')
                
                for QUESTION_ORDER, questao_row in enumerate(df_quest[df_quest._id.isin(id_questions)][['title','description','feedback','alternatives']].iterrows()):
                    QUESTION_ORDER=QUESTION_ORDER+1
                    print('Fazendo questão:', QUESTION_ORDER)
                    dict_questoes[f"Unidade_{LU_ORDER}"][f"Questao_{QUESTION_ORDER}"] = {}
                    dict_questoes[f"Unidade_{LU_ORDER}"][f"Questao_{QUESTION_ORDER}"]['title'] = from_html_to_text(questao_row[1]['title'])
                    dict_questoes[f"Unidade_{LU_ORDER}"][f"Questao_{QUESTION_ORDER}"]['description'] = from_html_to_text(questao_row[1]['description'])
                    dict_questoes[f"Unidade_{LU_ORDER}"][f"Questao_{QUESTION_ORDER}"]['feedback'] = from_html_to_text(questao_row[1]['feedback'])
                    dict_questoes[f"Unidade_{LU_ORDER}"][f"Questao_{QUESTION_ORDER}"]['alternatives'] = questao_row[1]['alternatives']
        #break


Requisitando curso: Ciências Contábeis - Disciplina: ANÁLISE DE CUSTOS
Fazendo LU: 1
Fazendo aula: 5 - 4 questões
Fazendo questão: 1
Fazendo questão: 2
Fazendo questão: 3
Fazendo questão: 4
Fazendo LU: 2
Fazendo aula: 5 - 5 questões
Fazendo questão: 1
Fazendo questão: 2
Fazendo questão: 3
Fazendo questão: 4
Fazendo questão: 5
Fazendo LU: 3
Fazendo aula: 5 - 4 questões
Fazendo questão: 1
Fazendo questão: 2
Fazendo questão: 3
Fazendo questão: 4
Fazendo LU: 4
Fazendo aula: 5 - 4 questões
Fazendo questão: 1
Fazendo questão: 2
Fazendo questão: 3
Fazendo questão: 4


In [49]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint='https://openai-dados-lab-poc.openai.azure.com/',
    api_key=os.environ["AZURE_OPENAI_API_KEY"]
)   


In [50]:
from langchain_core.documents import Document

for LU in dict_questoes.keys():
    for questao in dict_questoes[LU].keys():
        questao_texto = dict_questoes[LU][questao]['title']+dict_questoes[LU][questao]['description']+dict_questoes[LU][questao]['feedback']    
        print(LU, questao, questao_texto)
        
        vector_store = FAISS(
            embedding_function=embeddings,
            index=faiss.IndexFlatL2(len(embeddings.embed_query("hello world"))),
            docstore=InMemoryDocstore(),
            index_to_docstore_id={})
   
        vector_store.add_documents(
            documents=[Document(i) for i in dict_conteudo[LU].values()],
            ids=list(dict_conteudo[LU].keys()))
        
        results = vector_store.similarity_search(questao_texto, k=2)
        dict_questoes[LU][questao]['aula_relacionada_1'] = results[0].page_content
        dict_questoes[LU][questao]['aula_relacionada_2'] = results[1].page_content

Unidade_1 Questao_1 A transparência na gestão dos gastos indiretos também é essencial para a tomada de decisões informadas. Ao ter uma visão clara dos gastos associados a atividades administrativas e operacionais, os gestores podem desenvolver estratégias mais assertivas, identificar oportunidades de melhoria e antecipar possíveis desafios financeiros.O que abrange a Mão de Obra Indireta?A Mão de Obra Indireta engloba os custos relacionados a trabalhadores cujas atividades não podem ser diretamente associadas a produtos específicos. Isso inclui custos associados a supervisores, sendo, portanto, a alternativa d) correta.Salários dos operários:Esta alternativa está incorreta, pois os salários dos operários normalmente são considerados como parte da Mão de Obra Direta.Benefícios aos trabalhadores:Esta alternativa está incorreta, pois benefícios aos trabalhadores podem se referir tanto à Mão de Obra Direta quanto à Mão de Obra Indireta, dependendo da natureza dos benefícios.Custos diretame

In [51]:
from langchain_community.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint="https://openai-dados-lab-poc.openai.azure.com/",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_deployment="gpt-4o",
    openai_api_version="2024-02-01"
)

C:\Users\renan.vital\AppData\Local\Temp\ipykernel_28648\3739503124.py:3: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [52]:
for LU in dict_questoes.keys():
    for questao in dict_questoes[LU].keys():
        questao_texto = dict_questoes[LU][questao]['title']+dict_questoes[LU][questao]['description']+dict_questoes[LU][questao]['feedback']    
        print(LU, questao, questao_texto)
        
        curso_lista[0]
        template_inicial = \
            f"""System: Act as a Professor of {curso_lista[0]} - {disciplina}. Your role is develop a
            step by step guide to solve the question below, using two lectures. 
            Question:  {dict_questoes[LU][questao]['title']+dict_questoes[LU][questao]['description']}.
            Correct Answer: {dict_questoes[LU][questao]['feedback']}.
            ----------------------------------
            Context: Use these two lectures below to create the guide: 
            {dict_questoes[LU][questao]['aula_relacionada_1'].split('=|>')[0]}: {dict_questoes[LU][questao]['aula_relacionada_1'].split('=|>')[1]}.
            {dict_questoes[LU][questao]['aula_relacionada_2'].split('=|>')[0]}: {dict_questoes[LU][questao]['aula_relacionada_2'].split('=|>')[1]}.
            ----------------------------------
            Action: Briefly sumarize in bullet points the main subjects of each lecture, necessary to solve the question.
            Rule: Completion must contain just the main topics and its sub-topics.
            Topics are the lectures and its respective Unit, while sub-topics are the important subjects to solve the question.
            Example:
            - Unidade 1 Aula 1:\n Understand the theory of ..."""

        completion_inicial = llm.invoke(template_inicial)
        dict_questoes[LU][questao]['roteiro_questao'] = completion_inicial.content


Unidade_1 Questao_1 A transparência na gestão dos gastos indiretos também é essencial para a tomada de decisões informadas. Ao ter uma visão clara dos gastos associados a atividades administrativas e operacionais, os gestores podem desenvolver estratégias mais assertivas, identificar oportunidades de melhoria e antecipar possíveis desafios financeiros.O que abrange a Mão de Obra Indireta?A Mão de Obra Indireta engloba os custos relacionados a trabalhadores cujas atividades não podem ser diretamente associadas a produtos específicos. Isso inclui custos associados a supervisores, sendo, portanto, a alternativa d) correta.Salários dos operários:Esta alternativa está incorreta, pois os salários dos operários normalmente são considerados como parte da Mão de Obra Direta.Benefícios aos trabalhadores:Esta alternativa está incorreta, pois benefícios aos trabalhadores podem se referir tanto à Mão de Obra Direta quanto à Mão de Obra Indireta, dependendo da natureza dos benefícios.Custos diretame

In [ ]:
aulas_roteiro=''
for LU in dict_questoes.keys():
    for questao in dict_questoes[LU].keys():
        aulas_roteiro = f"{aulas_roteiro} \n - {dict_questoes[LU][questao]['roteiro_questao']} \n"    
aulas_roteiro

In [ ]:
aulas_roteiro=''
for LU in dict_questoes.keys():
    for questao in dict_questoes[LU].keys():
        aulas_roteiro = f"{aulas_roteiro} \n - {dict_questoes[LU][questao]['roteiro_questao']} \n"    

template_inicial = \
    f"""System: Act as a Professor of {curso_lista[0]} - {disciplina}. Your role is develop a
    step by step guide to study for an exam. Use the guide provided below: \n {aulas_roteiro}. 
    ----------------------------------
    Action: Create a friendly step by step guide.
    Important:
    - Topics should sorted as this example between triple backticks: ```Unidade 1 - AULA 1, Unidade 1 - AULA 2, Unidade 2 - AULA 1, Unidade 3 - AULA 1```;
    - Completion must contain just the guide;
    - Completion must be in brazilian portuguese;"""

completion_inicial = llm.invoke(template_inicial)
        
completion_inicial.content

In [11]:
aulas_roteiro=''
for LU in dict_questoes.keys():
    for questao in dict_questoes[LU].keys():
        aulas_roteiro = f"{aulas_roteiro} \n - {dict_questoes[LU][questao]['roteiro_questao']} \n"    

template_inicial = \
    f"""System: Act as a Professor of {curso_lista[0]} - {disciplina}. Your role is develop a
    step by step guide to study for an exam. Use the guide provided below: \n {aulas_roteiro}. 
    ----------------------------------
    Action: Create a friendly step by step guide.
    Important:
    - Topics should sorted as this example between triple backticks: ```Unidade 1 - AULA 1, Unidade 1 - AULA 2, Unidade 2 - AULA 1, Unidade 3 - AULA 1```;
    - Completion must contain just the guide;
    - Completion must be in brazilian portuguese;"""

completion_inicial = llm.invoke(template_inicial)
        
completion_inicial.content

'```Unidade 1 - AULA 1```\n1. **Inovação e Tecnologia na Comunicação**:\n   - **Realidade aumentada**: Oferece experiências imersivas e interativas. (Apoia a afirmativa I como correta)\n   - **Abordagens éticas**: Discussão sobre a necessidade de abordagens éticas na comunicação digital. (Relaciona-se com a afirmativa IV, indicando que a privacidade pode ser gerenciada, tornando a afirmativa IV incorreta)\n   - **Interatividade**: Importância da interatividade e transformação do público de receptor a coautor. (Contraria a afirmativa II, que diz que mídias sociais diminuem a importância do conteúdo personalizado)\n\n```Unidade 1 - AULA 4```\n2. **O Futuro da Comunicação no Brasil**:\n   - **Adaptabilidade**: Importância da adaptabilidade às tendências emergentes e inovações tecnológicas. (Apoia a afirmativa I)\n   - **Inteligência Artificial**: Uso de inteligência artificial para análise de dados e segmentação de público. (Apoia a afirmativa III como correta)\n   - **Desafios éticos**: 

In [12]:
template_final = \
    f"""System: Act as a Professor of {curso_lista[0]} - {disciplina}. You are developing a study guide for your exam.
    This guide must contain:
    - Motivational paragraph: Congratulate the discipline conclusion. Remember how important is this knowledge for his carrer (length: 4 lines). 
    - Instructions:  Tell this guide was build based on meta cognition techniques, and it is a important tool that identifies key subjects on each class that will lickly be asked on the exam. The student should focus and comprehend each topic to perform well.
    - Guide: {completion_inicial.content}
    - Tips paragraph: Remember to sleep well, drink water before the exam, study on a quiet place and to avoid distractions from smartphones or internet.
    - Conclusion: Wish lucky to your student, be kind and positive. 
    ----------------------------------
    Important:
    - Be kind, positive and motivational;
    - Completion must contain the final versio. Therefore, do not need to tell what the paragraf is about;
    - Completion must be in brazilian portuguese;"""
completion_final = llm.invoke(template_final)
        
completion_final.content

'Parabéns pela conclusão da disciplina de Publicidade e Propaganda - Sistemas Brasileiros de Comunicação! Este é um passo importante em sua carreira, pois o conhecimento adquirido aqui será fundamental para navegar e se destacar no mundo dinâmico e em constante evolução da comunicação. Continue se dedicando e aplicando tudo o que aprendeu, pois o sucesso está ao seu alcance!\n\nEste guia foi construído com base em técnicas de metacognição e é uma ferramenta essencial para identificar os principais assuntos abordados em cada aula que provavelmente serão perguntados no exame. Foque e compreenda cada tópico para obter um bom desempenho.\n\n```Unidade 1 - AULA 1```\n1. **Inovação e Tecnologia na Comunicação**:\n   - **Realidade aumentada**: Oferece experiências imersivas e interativas. (Apoia a afirmativa I como correta)\n   - **Abordagens éticas**: Discussão sobre a necessidade de abordagens éticas na comunicação digital. (Relaciona-se com a afirmativa IV, indicando que a privacidade pode

In [29]:
for i in dict_questoes.keys():
    print(i)
    for j in dict_questoes[i]:
        print(j)
        for k in dict_questoes[i][j]:
            #print(k)
            if k=='aula_relacionada_1':
                print('aula_relacionada_1', dict_questoes[i][j]['aula_relacionada_1'])
            elif k=='aula_relacionada_2':
                print('aula_relacionada_2', dict_questoes[i][j]['aula_relacionada_2'])
                print('-'*100)

Unidade_1
Questao_1
aula_relacionada_1 Unidade_1 - AULA_1 =|> Inovação e Tecnologia na Comunicação: Nesta videoaula, exploraremos como os avanços tecnológicos remodelam o cenário da mídia, introduzindo novas formas de comunicação digital e interatividade. Discutiremos os impactos dessas mudanças na sociedade e os desafios emergentes na era da informação, enfatizando a necessidade de abordagens éticas. Esses conteúdos são fundamentais para sua formação, preparando-o para navegar e influenciar o futuro da comunicação no Brasil. Convido você a se juntar a nós nesta jornada de aprendizado.Ponto de Partida: A comunicação está em constante evolução, impulsionada por inovações tecnológicas que transformam a maneira como interagimos com o mundo ao nosso redor. Esta aula abordará o impacto desses avanços na mídia e a emergência de novas plataformas de comunicação digital, enfatizando a interatividade e os desafios que acompanham a era da informação. Como essas transformações afetam nossa socied

In [30]:
print(completion_inicial.content)

```Unidade 1 - AULA 1```
1. **Inovação e Tecnologia na Comunicação**:
   - **Realidade aumentada**: Oferece experiências imersivas e interativas. (Apoia a afirmativa I como correta)
   - **Abordagens éticas**: Discussão sobre a necessidade de abordagens éticas na comunicação digital. (Relaciona-se com a afirmativa IV, indicando que a privacidade pode ser gerenciada, tornando a afirmativa IV incorreta)
   - **Interatividade**: Importância da interatividade e transformação do público de receptor a coautor. (Contraria a afirmativa II, que diz que mídias sociais diminuem a importância do conteúdo personalizado)

```Unidade 1 - AULA 4```
2. **O Futuro da Comunicação no Brasil**:
   - **Adaptabilidade**: Importância da adaptabilidade às tendências emergentes e inovações tecnológicas. (Apoia a afirmativa I)
   - **Inteligência Artificial**: Uso de inteligência artificial para análise de dados e segmentação de público. (Apoia a afirmativa III como correta)
   - **Desafios éticos**: Desafios ét

In [13]:
print(completion_final.content)

Parabéns pela conclusão da disciplina de Publicidade e Propaganda - Sistemas Brasileiros de Comunicação! Este é um passo importante em sua carreira, pois o conhecimento adquirido aqui será fundamental para navegar e se destacar no mundo dinâmico e em constante evolução da comunicação. Continue se dedicando e aplicando tudo o que aprendeu, pois o sucesso está ao seu alcance!

Este guia foi construído com base em técnicas de metacognição e é uma ferramenta essencial para identificar os principais assuntos abordados em cada aula que provavelmente serão perguntados no exame. Foque e compreenda cada tópico para obter um bom desempenho.

```Unidade 1 - AULA 1```
1. **Inovação e Tecnologia na Comunicação**:
   - **Realidade aumentada**: Oferece experiências imersivas e interativas. (Apoia a afirmativa I como correta)
   - **Abordagens éticas**: Discussão sobre a necessidade de abordagens éticas na comunicação digital. (Relaciona-se com a afirmativa IV, indicando que a privacidade pode ser ger